Данный скрипт предназначен для вычисления ряда акустических параметров пения птиц. На вход подаётся директория, содержащая логи с данными, полученные из приложения Raven Ligth (Selection Tables). На выходе получаются 2 файла (csv и xls), содержащие следующие параметры каждого лога: длительность, общее кол-во песен, размер репертуара, медианную длительность песен, медианную длительность пауз между песнями, частоту смены напева, Slin, Scons, RE1, versatility.

In [1]:
import pandas as pd
import numpy as np
import math
import statistics
import os

In [2]:
# Здесь необходимо прописать рабочую директорию в формате 'C:\\Users\\..' 
# В данном конкретном случае os.getcwd() указывает на текущую директорию.

home = os.getcwd() 

In [3]:
data_files = []   
for i in os.listdir(home):
    if 'txt' in i:
        data_files.append(i)

# Ищем файлы с логами в директории. Т.к. скрипт написан под Selection Tables от Raven Ligth, то и поиск идёт по расширению txt.

In [4]:
# Преобразуем найденные файлы в датафреймы и соберём в единый список.

data_files_df = []

for file in data_files:
    df = pd.read_csv(file, sep = '\t')
    name = file.split('.')[0]
    df['name'] = name
    data_files_df.append(df)

In [5]:
def newname(x):
    x = x.lower()
    return x
    # функция для перевода наименования в нижний регистр    

In [6]:
results = pd.DataFrame() # создадим пустой датафрейм, куда будут добавляться параметры по каждому логу

for df in data_files_df:
    df = df.rename(columns = {'Begin Time (s)':'begin_time', 'End Time (s)':'end_time', 'Low Freq (Hz)':'low_freq', 'High Freq (Hz)':'high_freq', 'Delta Time (s)':'delta_time',  'Delta Freq (Hz)':'delta_freq', 'Avg Power Density (dB FS/Hz)':'power'}).rename(columns = newname)

    total_songs = df.shape[0]


    trans = df.annotation.to_list() # переводим столбец с ЕР в список, дальше итерируемся по нему 
    total = 0
    total_trans = (total_songs-1) # df.shape[0] - кол-во строк, т.е. наблюдений. Всего переходов = кол-во наблюдений-1
    for i in range(len(trans)-1):
        if trans[i] != trans[i+1]:
            total += 1             # переменная total отображает кол-во переходов к ЕР другого типа
    switching_rate = total / total_trans 

    rep_size = df.annotation.nunique() # размер репертуара (кол-во разных типов ЕР)

    df1 = df[['selection', 'annotation']] # сохраним новую таблицу для расчётов в отдельной переменной

    df1['annotation2'] = df.annotation.shift(-1) # для подсчёта переходов создадим второй столбец со сдвигом

    df1 = df1.dropna()
    # уберём последнее значение как пропущенное, сохраним новую таблицу в отдельной переменной

    df1.annotation2 = df1.annotation2.astype(int) # поправим тип переменной - уберём нули

    transactions = df1.groupby(['annotation', 'annotation2'], as_index = False).agg({'selection' : 'count'})\
    .pivot(index='annotation', columns='annotation2', values='selection' ).fillna(0) # получаем матрицу переходов

    n = transactions.shape[0] # размерность матрицы
    # если у нас матрица не квадратная (т.е. для первого ЕР нет на него перехода), 
    # то необходимо искусственно добавить лишний столбец к матрице, иначе будет ошибка.
    if transactions.shape[0] != transactions.shape[1]:  
        transactions.insert(0, "0", 0)

    matrix = transactions.to_numpy() # сохраним датафрейм с переходами как матрицу

    count = 0
    for i in range(n):
        for j in range(n):
            if matrix[i][j] != 0:
                count += 1 # вычисляем кол-во типов переходов
    SLIN = rep_size / count

    sum_freq_trans = 0

    for l in matrix.tolist():
        sum_freq_trans += max(l)  # ∑ наиболее частых переходов (для каждого типа ЕР)

    SCONS = sum_freq_trans / total_trans

    E0  = -rep_size**2*(1/rep_size * np.log2(1/rep_size))  # формула Шеннона для E0 

    # Фукция для расчёта переходных вероятностей

    def transition_matrix(transitions):
        n = 1+ max(transitions) #number of states

        M = [[0]*n for _ in range(n)]

        for (i,j) in zip(transitions,transitions[1:]):
            M[i][j] += 1

        #now convert to probabilities:
        for row in M:
            s = sum(row)
            if s > 0:
                row[:] = [f/s for f in row]
        return M

    m = transition_matrix(trans)  # Матрица переходных вероятностей

    E1 = 0
    for i in range(n):
        for j in range(n):
            if m[i][j] != 0:
                E1 += m[i][j]*np.log2(m[i][j]) # вычисляем Е1 по формуле Шеннона для каждой вероятности, не равной 0. 
    E1 = -E1  
    if rep_size > 1:  # В случае, если у нас всего 1 тип песни, при расчёте RE1 по стандартной формуле мы получим NaN.
        RE1=E1/E0  
    else:
        RE1=-E0   # Поэтому добавляем условие, и если репертуар состоит всего из одной песни, RE1 считаем равным Е1.

    if len(trans) >= 10:
        div = []
        for i in range(0, (len(trans) - len(trans)%10), 10): # создаём цикл от начала до последней десятки последовательности с шагом 10
            div.append(len(set(trans[i:i+10])))
        Vers = statistics.median(div)
    else:
        Vers = len(set(trans))

    fon_length = df.end_time.iloc[-1] # общая длительность фонограммы

    song_rate = df.shape[0] / (fon_length / 60)  # частота пения

    df['pause_length'] = df.begin_time.shift(-1) - df.end_time # добавляем колонку с длительностью пауз

    time_param = df[['delta_time', 'pause_length']].describe() # рассчитваем временнЫе параметры

    median_lenth = time_param.delta_time.loc['50%']
    median_pause = time_param.delta_time.loc['50%']

    result = pd.DataFrame({'name': df.name.iloc[0], 'total_lenght': fon_length.round(2), 'total_songs': total_songs, 'rep_size':rep_size, 'median_song_lenth':median_lenth.round(2),
                           'median_pause_length':median_pause.round(2), 'song_rate':song_rate.round(2), 'switching_rate':round(switching_rate, 2),
                          'SLIN':round(SLIN, 2), 'SCONS':round(SCONS, 2), 'RE1':RE1.round(2), 'versatility':Vers}, index=[0])

    results = results.append(result, ignore_index=True)

C:\Users\Vaanry\AppData\Local\Temp\ipykernel_11260\1266830014.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['annotation2'] = df.annotation.shift(-1) # для подсчёта переходов создадим второй столбец со сдвигом
C:\Users\Vaanry\AppData\Local\Temp\ipykernel_11260\1266830014.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.annotation2 = df1.annotation2.astype(int) # поправим тип переменной - уберём нули
C:\Users\Vaanry\AppData\Local\Temp\ipykernel_11260\1266830014.py:108: FutureWarning: The fr

In [7]:
results # ну посмотрим, что получилось :)

,name,total_lenght,total_songs,rep_size,median_song_lenth,median_pause_length,song_rate,switching_rate,SLIN,SCONS,RE1,versatility
0,MZ000027,14.87,4,1,0.51,0.51,16.15,0.00,1.00,1.00,0.00,1.0
1,MZ000037,105.07,21,10,0.90,0.90,11.99,0.85,0.59,0.60,0.14,6.5
2,MZ000073,660.04,123,8,0.80,0.80,11.18,0.84,0.28,0.45,0.39,4.0
3,MZ000077,63.77,11,1,1.06,1.06,10.35,0.00,1.00,1.00,0.00,1.0
4,MZ000092,352.29,37,5,0.85,0.85,6.30,0.56,0.33,0.53,0.50,3.0
5,MZ000123,494.38,25,3,0.77,0.77,3.03,0.12,0.60,0.92,0.03,2.0


In [8]:
results.to_csv('results.csv') # сохраняем результаты в папку в двух форматах.
results.to_excel('results.xlsx')